# Final Project Machine Learning with Probabilistic Programming 
## Fall 2018, Columbia University, New York, Prof. Alp Kucukelbir
### Jan-Felix Schneider (jfs2160), Brian Allen (ba xxxx) 



# Introduction 

In our final project, we would like to understand how certain phenomena (i.e. diseases, behavior, tweets, news, etc.) spread through an network of nodes (cities, people, twitter accounts, news pages etc.). In many cases one does only observe the time when a node in the network got "infected", but not from which previously infected node the newly infected node recieved the contagion. 

In this project, we would like to infer an underlying "influence network" from a set of influence cascades. The cascade data consists out of a timestamp, when the node was converted. We're building our efforts on the work of Rodriguez et al (2011), who build a probability model on the independent cascade model developed by Kempe et al (2003). 


#### Probabilisitc Model Decription 

For our model, we will use the following variables: 

$N$: number of nodes in graph <br>
$e_{i,j}$: directed edge between node $i$ and $j$  <br>
$\gamma_i > 0 $: susceptability of node i <br>
$t_i > 0 $: time elapsed since start of the cascade when node i was infectede <br>
$\beta_{i} \in \mathbb{R}^k$: polarity of a topic vector of size k of node i <br>
$\tau_{c} \in \mathbb{R}^k$: topic of a cascade c <br>
$\alpha_{i,j} \geq 0 $: strenght of influence between node i and j (directed) <br>
$p_{i,j}(\alpha_{i,j},\gamma_i,\tau_c,\beta_i,t_i, t_j)$: probability that node i influences node j <br>
$p_{i,j} = \sigma(\beta_i^T\tau_c \cdot \alpha_{i,j} \cdot \gamma_i)$: discuss additive or multiplicative behavior <br>
$I_t$: Matrix of infected nodes at time t


If a node gets infected it has the chance to infect its neighbors. 



#### Model Extention 

We extend the model in two ways: 
a) we implemented a generative processs and addede a prior to the influence parameters
b) we added topic parameter to the model 

a)


b) We believe, that the strength a node influences another node depends on the susceptibility of the node to the particular influence topic. For example, while a person may take advice from a friend in a certain area for example for career he or she might not be influence by advice on romantic issues. We modelled this phenomenon, by adding a prior to the influence between node, that depends on the topic preferences of the node and the topic of the cascade. 

#### Dataset

The dataset we are using is from the memetracker dataset. http://snap.stanford.edu/memetracker/index.html

To test the internal validity of our model, we generated a datset. 



In [4]:
from hdiffnet.preprocessing import Preprocessing
from hdiffnet.hdiffnet import ProbabilityModel

//anaconda/envs/py35tensorflow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


# Data Processing

We downloaded the memetracker cluster dataset and processed it. This dataset tracks when certain phrases have appeard at different webpages on the internet and the raw dataset includes a phrase, a phrase id, a related phrase cluster, the URL of a webpage and a timestamp. For our analysis, we included the top 200 webpages (to reduce the size of the dataset) and transformed the dataset, such that we assumed a phrase to be a cascade and normalized the timestamp to be seconds until the first phrase has been posted. 

In [5]:
# Load the data and transform it
p = Preprocessing("data/clust-qt08080902w3mfq5.txt")
p.preprocess_data(200)

,cascade_id,node_id,sec_till_start
40,23785.0,106,0.0
48,23785.0,63,9957.0
63,23785.0,143,31776.0
215,23785.0,83,136061.0
354,23785.0,1,231671.0
374,23785.0,1,237309.0
390,23785.0,1,242602.0
406,23785.0,1,247570.0
414,23785.0,1,252798.0
423,23785.0,1,257841.0


In [6]:
p.data.head()

,cascade_id,node_id,sec_till_start
40,23785.0,106,0.0
48,23785.0,63,9957.0
63,23785.0,143,31776.0
215,23785.0,83,136061.0
354,23785.0,1,231671.0


In [12]:
list(p.labels.keys())[:10]

['cbs11tv.com',
 'icscotland.icnetwork.co.uk',
 'money.cnn.com',
 'cnsnews.com',
 'miamiherald.com',
 'mercurynews.com',
 'kansascity.com',
 'thechronicleherald.ca',
 'icwolverhampton.icnetwork.co.uk',
 'nypost.com']

In [5]:
import itertools
data = data.sort_values('sec_till_start')
data = data.drop_duplicates(['cascade_id','node_id'])
data['sec_till_start'] = data.groupby('cascade_id')['sec_till_start'].transform(
            lambda x: (x - min(x)))

In [6]:
data['sec_till_start'] = data['sec_till_start']/3600

In [7]:
a = data.groupby('cascade_id')['node_id'].apply(list)
b = data.groupby('cascade_id')['sec_till_start'].apply(list)

In [8]:
result = []
for i in range(a.shape[0]):
    res = list(itertools.chain(*zip(a.iloc[i],b.iloc[i])))
    result.append(res)

In [9]:
max_T = data['sec_till_start'].max()

In [10]:
result[0]

[101,
 0.0,
 30,
 0.009444444444444445,
 14,
 0.0825,
 47,
 0.1025,
 3,
 0.10777777777777778,
 0,
 0.16916666666666666,
 84,
 0.20055555555555554,
 45,
 0.27055555555555555,
 1,
 0.45944444444444443,
 123,
 0.5005555555555555,
 58,
 0.8480555555555556,
 50,
 1.0227777777777778,
 9,
 1.08,
 2,
 1.1133333333333333,
 15,
 1.181111111111111,
 29,
 1.2147222222222223,
 13,
 1.2258333333333333,
 72,
 1.228611111111111,
 5,
 1.6194444444444445,
 94,
 1.8266666666666667,
 186,
 2.12,
 22,
 2.2330555555555556,
 115,
 2.547222222222222,
 164,
 2.703611111111111,
 28,
 2.8583333333333334,
 104,
 2.879722222222222,
 35,
 3.018611111111111,
 21,
 3.1844444444444444,
 40,
 4.363611111111111,
 34,
 4.393888888888889,
 89,
 4.4719444444444445,
 33,
 4.476111111111111,
 118,
 4.610277777777778,
 8,
 5.163611111111111,
 18,
 5.243611111111111,
 66,
 5.52,
 195,
 5.616944444444444,
 44,
 5.841388888888889,
 67,
 5.911388888888889,
 174,
 6.520555555555555,
 26,
 6.682777777777778,
 175,
 6.70277777777777

# Model 



### Implementing a generative process in TF and Edward

In our probabilistic model, a cascade if propagated in the following way. The observed variables is the time t_i, when a node is infected and we model it the following exponential distribution (REMARK: ADD THE POLARITY OF THE CASCADE): 

$t_i \sim \alpha_{i,j} exp\{-\alpha_{i,j}(t_i - t_j)\}$


The intutition behind this is, that when a node is infected, it can infect all its neighbors. The time when the neighboring node is infected, depends on the "strenght" of the relationship, which is indicated by alpha. A higher alpha leads to a faster infection time of the neighbor. 

The parameter $\alpha$ has the following prior: 

$\alpha_{i,j} \sim Gamma(\theta_1, \theta_2)$


Given a seed set $S$, which is a set of nodes that are infected initially, we can implemente this process for a graph in the following way: 
* For every edge in the adjacency matrix, draw a time $t_{i,j}| \alpha_{i,j}$
* To obtain $t_i$ from $t_{i,j}$, get the shortest-path for every node to a nodes from seed set $S$

We were able to implement this process in tensorflow to use an iterative approach, however when feeding this process to an Edward program we ran into the following problems... 

* A ...
* B ...

However, we used our generative process, to generate a dataset which we were then using to test the correctness of our inference algorithms, i.e. does our inference algorithm return the right parameters, for our proposed generative process. 

In [13]:
# Using the generative process, to generate a dataset

### Implementing the prior probability in TF 

As our implementation with edward did not return viable result, we were implementing a log-prior probability in tensorflow to use BFGS to obtain a MAP estimate. As the log-prior is convex, this approach should yield a good estimate, even if Variational inference methods, may have resulted in better estimates. (REMARK Discuss this! Is it true?)






In [14]:
# Feed the data into a tensorflow log-prior model.
model = ProbabilityModel(result[:500], 200, T = max_T)

NameError: name 'result' is not defined

# Inference

### Using edward Inference



Critique of the edward documentation!!!


### Using BFGS to obtain MAP estimates

We were using the Scipy BFGS optimizer to obtain our MAP results. 

In [12]:
%time a = model.map_estimate_BFGS(max_iter=1000)

INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 492283.625000
  Number of iterations: 324
  Number of functions evaluations: 366
CPU times: user 28min 37s, sys: 8min 2s, total: 36min 40s
Wall time: 11min 27s


10: 0:26 min 
20: 1:24 min
30: 1:28 min
50: 2:55 min
100: 6:55 min
200: 10:23 min
400: 20:27 min


In [17]:
a.round(2)

array([[0.51, 0.  , 0.  , ..., 0.01, 0.53, 0.68],
       [0.  , 0.62, 0.  , ..., 0.  , 0.  , 0.71],
       [0.  , 0.  , 0.71, ..., 0.  , 0.65, 0.55],
       ...,
       [0.  , 0.  , 0.  , ..., 0.62, 0.  , 0.59],
       [0.  , 0.  , 0.  , ..., 0.  , 0.61, 0.73],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.63]], dtype=float32)

# Criticism

It is not trivial to test the results of our inference model, as a specific cascade if very unlikely to occur and the resulting prediction problem has a very high dimensionality. That's why we used the following ways to see, whether our model describes the reality in a proper way: 

1. Compare cascade predictions with a heldout dataset
2. Compare our influence graph with the number of links between the website (a proxy for interconnectedness of websited) 
3. Visualize our influence graph to understand relationships 



### Predictions

#

### Visualizing MAP Estimate Results 

# References


* Kempe
* ...

